In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import os

import dro.utilities as dro

/home/justink/.conda/envs/braintv_behav/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


failed to import CV2 in core.utilities
failed to import mask in core.jcamdata.py
failed to import get_mask_dict_gui in core.epoch.py
failed to import aibs.Analysis.Core.smooth
failed to import mask relalated functions in __init__.py


In [26]:
pwd

u'/local1/Projects/braintv_behavior_piloting/notebooks'

In [27]:
%%bash
cp -Rupv /local1/Projects/braintv_behavior_piloting/data/raw/*mouse=*.pkl /local1/Projects/ExploringBehavior/data

`/local1/Projects/braintv_behavior_piloting/data/raw/170110131936-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M272464.pkl' -> `/local1/Projects/ExploringBehavior/data/170110131936-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M272464.pkl'
`/local1/Projects/braintv_behavior_piloting/data/raw/170110134920-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M271728.pkl' -> `/local1/Projects/ExploringBehavior/data/170110134920-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M271728.pkl'
`/local1/Projects/braintv_behavior_piloting/data/raw/170110135032-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M276951.pkl' -> `/local1/Projects/ExploringBehavior/data/170110135032-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M276951.pkl'
`/local1/Projects/braintv_behavior_piloting/data/raw/170110145711-task=DoC_MNIST_stage=0v1_probes_n=3_mouse=M258274.pkl' -> `/local1/Projects/ExploringBehavior/dat

In [28]:
# datapath = '/data/nc-ophys/Doug/All_DoC_Animals'
datapath = '/local1/Projects/ExploringBehavior/data'
df = dro.load_from_folder(datapath)

Loading /local1/Projects/ExploringBehavior/data/170126102817-task=DoC_NaturalImages_CAMMatched_n=8_stage=Testing_mouse=M271733.pkl
Loading /local1/Projects/ExploringBehavior/data/161214120353-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M258274.pkl
Loading /local1/Projects/ExploringBehavior/data/170113100257-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M276444.pkl
Loading /local1/Projects/ExploringBehavior/data/161207095541-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M273578.pkl
Loading /local1/Projects/ExploringBehavior/data/161117134401-task=DetectionOfChange_Gratings_stage=2_LickDetectTraining_mouse=M261584.pkl
error loading file 160401100723-227383.pkl: index out of bounds
Loading /local1/Projects/ExploringBehavior/data/170120121253-task=DoC_NaturalImages_n=8_stage=Testing_mouse=M273577.pkl
Loading /local1/Projects/ExploringBehavior/data/170109102611-task=DoC_MNIST_stage=0v1_probes_n=3_mouse=M272465.pkl
error loading file

In [29]:
len(df)

2092453

In [30]:
# tidy mouse IDs
df['mouse_id'] = df['mouse_id'].map(lambda s: s.split('_')[0])
df['mouse_id'] = df['mouse_id'].str.upper()
df['mouse_id'] = df['mouse_id'].map(lambda s: 'M{}'.format(s) if s.startswith('M')==False else s)
df = df[df['mouse_id']!='MTEST']
df.groupby('mouse_id').size()

mouse_id
M187476    36759
M203172    18479
M203173    14831
M214522    33580
M217744    13868
M218021    62088
M218307    30290
M219018    21955
M219290     8943
M219521    41102
M221772    45157
M223817     7315
M226336    19408
M226941    15107
M227383     7336
M227399    10951
M229747    73600
M230032    18423
M230344    75071
M230345    41269
M241435    27551
M241436    54849
M244779    29086
M246782    25591
M247884    29952
M248528    36970
M249123     9231
M249843    46216
M250783    38189
M250786    37011
           ...  
M258196    38567
M258274    48503
M258275    47487
M258276    44895
M258993    23565
M261584    64776
M265154    30864
M265158    26126
M265166    24142
M271723       83
M271728    18503
M271733    35897
M271966    42442
M272464    23720
M272465    55203
M273577    23867
M273578    24588
M275962    24376
M275963     1584
M275965     1290
M276444    24270
M276445    20425
M276600    27798
M276950    28881
M276951    30532
M277931     1585
M278229    19018
M2783

In [31]:
## identify training day

trim = np.vectorize(lambda x: x[:10])

training_day_lookup = {}

for key, group in df.groupby(['mouse_id',]):
    dates = np.sort(trim(group['startdatetime'].unique().astype(str)))
    
    training_day_lookup[key] = {date:training_day for training_day,date in enumerate(dates)}

df['date'] = df['startdatetime'].map(lambda x: str(x)[:10])
df['training_day'] = df.apply(lambda row: training_day_lookup[row['mouse_id']][row['date']],axis=1)

In [32]:
## build arrays for change detection

df['change'] = df['trial_type']=='go'
df['detect'] = df['response']==1.0

# ## unwrap the response window
# df['response_window_lower'] = df['response_window'].map(lambda x: x[0])
# df['response_window_upper'] = df['response_window'].map(lambda x: x[1])

In [33]:
df['change_time'] = df['change_time'].map(lambda x: np.inf if pd.isnull(x) else x)

In [34]:
# reaction time
df['reaction_time'] = df['lick_times'].map(lambda x: x[0] if len(x)>0 else np.inf)
df['reaction_time'] = df.apply(lambda row: row['reaction_time'] - row['change_time'],axis=1)

In [35]:
from behaviorist.metrics import *

def compute_metrics(x):
    result = {
        'd-prime': discrim(x,'change','detect',metric=d_prime),
#         'accuracy': discrim(x,'change','detect',metric=metrics.accuracy_score),
        'discrim_p': discrim(x,'change','detect',metric=discrim_p),
#         'response_bias': response_bias(x,'detect'),
#         'earned_water': earned_water(x), 
#         'total_water': total_water(x), 
        'num_trials': num_trials(x),
#         'reaction_time_50th%ile': reaction_times(x,percentile=50),
#         'reaction_time_05th%ile': reaction_times(x,percentile=5),
#         'reaction_time_95th%ile': reaction_times(x,percentile=95),
    }
    
    return pd.Series(result, name='metrics')

pdf = df.groupby(['mouse_id','training_day']).apply(compute_metrics).reset_index()
# pdf['reaction_time_spread'] = pdf['reaction_time_95th%ile'] - pdf['reaction_time_05th%ile']
pdf.head()

# pdf['d_prime'] = pdf['d-prime']
pdf['stimulus_control'] = pdf['discrim_p'] < 0.01

In [36]:
class RisingEdge():
    """
    This object implements a "rising edge" detector on a boolean array.
    
    It takes advantage of how pandas applies functions in order.
    
    For example, if the "criteria" column in the `df` dataframe consists of booleans indicating
    whether the row meets a criterion, we can detect the first run of three rows above criterion
    with the following
    
        first_run_of_three = (
            df['criteria']
            .rolling(center=False,window=3)
            .apply(func=RisingEdge().check)
            )
    
    ```
    
    """
    def __init__(self):
        self.firstall = False
    def check(self,arr):
        if arr.all():
            self.firstall = True
        return self.firstall

In [37]:

num_tr_thresh = 50
dprime_thresh = 0.5

consecutive_sessions = 2

pdf['criteria'] = (
    pdf['stimulus_control']
    & (pdf['num_trials'] > num_tr_thresh)
    & (pdf['d-prime'] > dprime_thresh)
)

mask = pdf.num_trials > num_tr_thresh # ignore sessions with very few trials

training_day_at_criterion = {}

for key, group in pdf.set_index('training_day').sort_index().groupby(['mouse_id',]):
#     print group.head()
    learned = (
        group['criteria']
        .rolling(center=False,window=consecutive_sessions)
        .apply(func=RisingEdge().check)
        )
    summary = pd.DataFrame(
        [learned.rename('learned'),
         group['stimulus_control'],
         group['d-prime'],
         group['num_trials'],
        ]).transpose()
    
    try:
        first_session = summary['learned'][summary['learned']==1.0].index[0]
    except IndexError:
        first_session = np.nan
    
    print key, first_session
#     print summary.tail()
    
    training_day_at_criterion[key]=first_session

M187476 4
M203172 1
M203173 1
M214522 1
M217744 1
M218021 36
M218307 36
M219018 2
M219290 1
M219521 24
M221772 1
M223817 nan
M226336 1
M226941 1
M227383 nan
M227399 nan
M229747 1
M230032 17
M230344 57
M230345 34
M241435 1
M241436 1
M244779 1
M246782 19
M247884 2
M248528 1
M249123 nan
M249843 1
M250783 9
M250786 9
M253178 1
M254268 1
M254270 1
M256238 1
M256240 13
M258084 1
M258089 11
M258173 22
M258174 4
M258194 6
M258196 8
M258274 5
M258275 8
M258276 37
M258993 1
M261584 10
M265154 nan
M265158 25
M265166 nan
M271723 nan
M271728 7
M271733 11
M271966 11
M272464 8
M272465 12
M273577 4
M273578 11
M275962 9
M275963 nan
M275965 nan
M276444 20
M276445 7
M276600 6
M276950 8
M276951 6
M277931 nan
M278229 3
M278329 nan
M278332 nan
M278463 nan


In [38]:
training_day = pd.DataFrame([{'mouse':mouse,'day':day} for mouse,day in training_day_at_criterion.iteritems()])

In [39]:
training_day['learned'] = training_day['day']>0

In [40]:
print training_day['learned'].sum()
print len(training_day['learned'])
print training_day['learned'].mean()

57
70
0.814285714286


In [41]:
from braintv_pilot.cohorts import cohort_assignment
cohort_assignment

,mouse,experiment,cohort,day_zero
0,M272464,visual_behavior_development,4,2016-10-31
1,M271733,visual_behavior_development,4,2016-10-31
2,M271728,visual_behavior_development,4,2016-10-31
3,M276951,visual_behavior_development,4,2016-10-31
4,M276950,visual_behavior_development,4,2016-10-31
5,M276600,visual_behavior_development,4,2016-10-31
6,M272465,visual_behavior_development,3,2016-10-10
7,M265158,visual_behavior_development,3,2016-10-10
8,M271966,visual_behavior_development,3,2016-10-10
9,M265154,visual_behavior_development,3,2016-10-10


In [43]:
training_day = training_day.merge(cohort_assignment,on='mouse',how='left')

In [44]:
ignore = training_day['cohort']==6
ignore.sum()

6

In [70]:
print training_day[~ignore]['learned'].sum()
print len(training_day[~ignore]['learned'])
print training_day[~ignore]['learned'].mean()

57
64
0.890625


In [47]:
old_training_day_at_criterion= {"M187476": 4,
"M203172": 1,
"M203173": 1,
"M214522": 1,
"M217744": 1,
"M218021": 36,
"M218307": 36,
"M219018": 3,
"M219290": 1,
"M219521": 24,
"M221772": 1,
"M223817": None,
"M226336": 1,
"M226941": 1,
"M227383": None,
"M227399": None,
"M229747": 1,
"M230032": 17,
"M230344": 57,
"M230345": 34,
"M241435": 1,
"M241436": 1,
"M244779": 1,
"M246782": 19,
"M247884": 2,
"M248528": 1,
"M249123": None,
"M249843": 1,
"M250783": 9,
"M250786": 9,
"M253178": 1,
"M254268": 1,
"M254270": 1,
"M256238": 1,
"M256240": 13,
"M258084": 1,
"M258089": 11,
"M258173": None,
"M258174": 4,
"M258196": 8,
"M258274": 5,
"M258275": 8,
"M258276": None,
"M258993": 1,
"M261584": 10,
    }

In [55]:
from braintv_pilot.cohorts import mouse_info

In [56]:
mouse_df = training_day['mouse'].apply(lambda mouse: pd.Series(mouse_info(mouse)))

In [58]:
training_day = training_day.merge(
    mouse_df,
    how='left',
    left_on='mouse',
    right_on='mouse_id'
)

In [71]:
training_day[~ignore].groupby('group')['learned'].agg([np.mean,len])

,mean,len
group,,
Ai93,1.000000,1
C57BL/6J,0.000000,2
Cux2-CreERT2,1.000000,1
Cux2-CreERT2; Camk2a-tTA;Ai93,1.000000,1
EMX-Cre-Ai35,1.000000,5
EmX-IRES-Cre,1.000000,1
Emx-IRES-Cre;Ai95(NC),1.000000,1
Emx-IRES-Cre;Ai96(NC),1.000000,1
Emx1-IRES-Cre; Camk2a-tTA; Ai93,1.000000,1


In [72]:
training_day[training_day.cohort==6]['group']

4          Ntsr1-Cre; Camk2a-tTa; Ai93 (NC)
8          Ntsr1-Cre; Camk2a-tTa; Ai93 (NC)
13    Rorb-IRES2-Cre; Camk2a-tTA; Ai93 (NC)
21    Rorb-IRES2-Cre; Camk2a-tTA; Ai93 (NC)
24                                    nr5a1
60    Rorb-IRES2-Cre; Camk2a-tTA; Ai93 (NC)
Name: group, dtype: object